# LoadinG The Data

In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import math

#using the preprocess Sqlite Table to read the  data
con = sqlite3.connect('./amazon-fine-food-reviews/finalsqlite')

#Filtering only positive review
positive_review = pd.read_sql_query("""
SELECT * FROM Reviews Where Score == 'positive'
""", con)

#Extracting 2000 Positive review Randomly
positive_smaller_review_set = positive_review.sample(200)
print(positive_smaller_review_set.shape)

#Filtering only negative review
negative_review = pd.read_sql_query("""
SELECT * FROM Reviews Where Score == 'negative'
""", con)

#Extracing 2000 random negative review
negative_smaller_review_set = negative_review.sample(200)
print(negative_smaller_review_set.shape)

#Concanating Negative and Positive review into one dataframe
frames = [positive_smaller_review_set, negative_smaller_review_set]
final = pd.concat(frames)

#Sort the dataframe data into the increasing Time order
final = final.sort_values(by=['Time'])
print(final.head)
print(final.shape)


C:\Users\DANAV\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(200, 12)
(200, 12)
<bound method NDFrame.head of          index      Id   ProductId          UserId  \
9023    155416  168562  B0001ES9F8  A1R9HPMUY9DGVE   
51      417904  451926  B00004CXX9  A31V5BH5YHG4TZ   
3884    437963  473617  B0007V1Q7I  A2Q261K2VVCDHE   
12264   356387  385494  B0002994Q0  A1YZ5CHHCLP8Y5   
51113    65409   71054  B000F4D5JE  A2E3WMF9RWW2X2   
31621   362365  391888  B000CQBZPG  A1AJ5JVIO3PD25   
14367    76105   82818  B0002PHDYO  A1VPUK22B4MTDE   
47456   254362  275779  B000EVLS2O  A2BKC69F4BRO29   
43469    49112   53347  B000EMM9YY  A25HD1WPIAYPLQ   
195209  489631  529434  B0029NIF44   AGJLD33CGEXXL   
5582    218028  236346  B000CONM8Q  A10CMEPEV19WAB   
223108  208660  226130  B0036X748E  A2NPF2TZWXX5RT   
95646   401783  434432  B000P8ZNOK  A1NFYVLLYNX48R   
164268   91411   99442  B001FA1KU8  A2CB7HX0U1CUKW   
149578  408767  442095  B001EO5MZY   AV50PSPVKS3RD   
19007   323302  349941  B0006J32A0  A18GY5MOJU8KK6   
151852  184973  200653  B001EO5U

# Bag OF word(BOW)

In [2]:
#BOW of preprocess text
count_vect = CountVectorizer()
final_counts_preprocess_text = count_vect.fit_transform(final['CleanedText'].values)

print("Shape of vector of Preprocess review(text) after BOW(unigram)",final_counts_preprocess_text.get_shape())

Shape of vector of Preprocess review(text) after BOW(unigram) (400, 2806)


# Standariztion Of Vector

In [3]:
from sklearn.preprocessing import StandardScaler

#Standarization OF review text
standarized_data = StandardScaler().fit_transform(final_counts_preprocess_text.toarray()) 

C:\Users\DANAV\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


 # Train and Test Split

In [7]:
# create design matrix X and target vector y

X = np.array(standarized_data[:,:])  
y = np.array(final['Score'].values)
print(X.shape)
print(y.shape)
# Calculating the train and test size and dividing the data into train and test chunks into the 70 - 30 ratio
train_size = math.floor(X.shape[0] * 0.7)

#split the data set into train and test
X_train = X[: train_size ,:]
X_test  = X[train_size : , :]  
y_train = y[: train_size ]
y_test =  y[train_size : ]

(400, 2806)
(400,)


# Grid Serach On Bow

In [9]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

tuned_parameters =[{'C' : [10 **-4, 10**-2, 10**0, 10**2, 10**4]}]

model = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'f1', cv=5)
model.fit(X_train, y_train)
print(model.best_estimator_)
# print(model.score(X_test, y_test)) 

C:\Users\DANAV\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1030: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['negative', 'positive'], dtype='<U8')

# Tf-Idf

In [11]:
#tf-idf of preprocess text
tf_idf_vect  = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf_preprocess_text =  tf_idf_vect.fit_transform(final['Text'].values)

#Shape of  vectors
print("Shape of vector of Preprocess review(text) after Tf-Idf",final_tf_idf_preprocess_text.get_shape())

Shape of vector of Preprocess review(text) after Tf-Idf (100, 7274)


# Standariztion Of Vector

In [12]:
from sklearn.preprocessing import StandardScaler

#Standarization OF review text
standarized_data = StandardScaler().fit_transform(final_tf_idf_preprocess_text.toarray())

# knn on TF-IDF vector 

In [13]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on Tf-Idf using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on Tf-Idf using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf using K_D tree approach for k = 21 is 53.333333%


# Average Word2Vec

In [17]:
#Definition oF CleanHtml and CleanPunc for Preprocessing the reviw text
import re
import gensim

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

# Train your own Word2Vec model using your own text corpus

i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
    
   

In [18]:
# Train the model
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=1,size=50, workers=4)
                                 
Avg_sent_vectors = []
for sent in list_of_sent:
    sent_vect = np.zeros(50)
    count = 0
    for word in sent:
        try :
            sent_vect += w2v_model.wv[word]
            count = count + 1
        except:
            pass
    
    sent_vect = sent_vect / count
    Avg_sent_vectors.append(sent_vect)
     
#Standarization Of Avg Word2Vec vector
standardized_data = StandardScaler().fit_transform(Avg_sent_vectors)
print(standardized_data.shape)



(100, 50)


# knn on Average Word2Vec vector 

In [19]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on Word2vec using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on Word2vec using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Word2vec using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Word2vec using K_D tree approach for k = 21 is 53.333333%


# Tf-Idf Word2vec

In [21]:
tf_idf_feat = tf_idf_vect.get_feature_names()
tfidf_sent_vect=[]
row= 0;
for sent in list_of_sent:
    sent_vec = np.zeros(50)
    weight_sum = 0
    for word in sent:
        try:
            if((word in  w2v_model.wv.vocab) and (word in tf_idf_feat)):
                w_vec = w2v_model.wv[word]
                tf_idf_vec = final_tf_idf[row,tf_idf_feat.index(word)]
                sent_vec = sent_vec + (w_vec*tf_idf_vec)
                weight_sum = weight_sum + tf_idf_vec
        except:
            pass
    sent_vec = sent_vec/weight_sum
    tfidf_sent_vect.append(sent_vec)
    row= row+1
    
# Removing NAN values    
tfidf_sent_vect_values = np.nan_to_num(tfidf_sent_vect)

#Standarization Of Tf-Idf Word2Vec Vector
standardized_data = StandardScaler().fit_transform(tfidf_sent_vect_values)
print(standardized_data.shape)



C:\Users\DANAV\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


(100, 50)


# knn on Tf-Idf Word2Vec vector 

In [22]:
# Using knn_accuracy Function finding optimal k and accuracy on that optimal k

optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'brute')
print('\nThe accuracy of the knn classifier on Tf-Idf Word2vec using Brute Force approach for k = %d is %f%%' % (optimal_k, acc))
optimal_k, acc =  knn_accuracy(standarized_data , algorithms = 'kd_tree')
print('\nThe accuracy of the knn classifier on Tf-Idf Word2vec using K_D tree approach for k = %d is %f%%' % (optimal_k, acc))

brute
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf Word2vec using Brute Force approach for k = 21 is 53.333333%
kd_tree
[0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5267857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.5142857142857142, 0.4708333333333333, 0.5, 0.5125, 0.4976190476190475, 0.4976190476190476]

The optimal number of neighbors is 21.

The accuracy of the knn classifier on Tf-Idf Word2vec using K_D tree approach for k = 21 is 53.333333%
